In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import plotly.express as px

%matplotlib notebook

In [2]:
file_dir = '../../Data/parking_lot/'
filenames =[
            '10552_NYU_M2 - Scanner 1 - 190511_164039_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_164239_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_164445_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_172558_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_172753_1 - originalpoints.laz',    
            '10552_NYU_M2 - Scanner 1 - 190511_172928_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_180428_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_180632_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_180819_1 - originalpoints.laz',
            '10552_NYU_M3 - Scanner 1 - 190511_200348_1 - originalpoints.laz',
            '10552_NYU_M3 - Scanner 1 - 190511_200600_1 - originalpoints.laz',
            '10552_NYU_M3 - Scanner 1 - 190511_200742_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_163824_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_164640_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_172416_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_173110_1 - originalpoints.laz',
            '10552_NYU_M2 - Scanner 1 - 190511_164845_1 - originalpoints.laz',
           '10552_NYU_M2 - Scanner 1 - 190511_172201_1 - originalpoints.laz',
           '10552_NYU_M2 - Scanner 1 - 190511_173238_1 - originalpoints.laz',
           '10552_NYU_M3 - Scanner 1 - 190511_200938_1 - originalpoints.laz',
           '10552_NYU_M3 - Scanner 1 - 190511_200212_1 - originalpoints.laz',
           '10552_NYU_M2 - Scanner 1 - 190511_181004_1 - originalpoints.laz',
           '10552_NYU_M2 - Scanner 1 - 190511_180231_1 - originalpoints.laz'
 ]


pt_files = [   'las_points_163824.lz',
                'las_points_164039.lz',
                'las_points_164239.lz',
                'las_points_164445.lz',
                'las_points_164640.lz',
                'las_points_164845.lz',
                'las_points_172201.lz',
                'las_points_172416.lz',
                'las_points_172558.lz',
                'las_points_172753.lz',
                'las_points_172928.lz',
                'las_points_173110.lz',
                'las_points_173238.lz',
                'las_points_180231.lz',
                'las_points_180428.lz',
                'las_points_180632.lz',
                'las_points_180819.lz',
                'las_points_181004.lz',
                'las_points_200212.lz',
                'las_points_200348.lz',
                'las_points_200600.lz',
                'las_points_200742.lz',
                'las_points_200938.lz',]

# Corresponds to LAS 1.2 Point Data Record Format 1
columns_dublin_pt_cloud = [
    'X',
    'Y',
    'Z',
    'intensity',
    'return_number_byte',
    'classification_byte',
    'scan_angle',
    'user_data',
    'pt_src_id',
    'gps_time']

columns_point_cloud = [
    'X','Y','Z',
    'intensity',
    'flag_byte',
    'classification_flags',
    'classification_byte',
    'user_data',
    'scan_angle',
    'pt_src_id',
    'gps_time']

In [3]:
def raw_to_df(raw,column_names):
    '''function takes raw output of laspy.File.get_points() and column names, and returns a pandas Dataframe'''
    raw_list = [a[0].tolist() for a in raw]
    df = pd.DataFrame(raw_list,columns = column_names)
    return df

def scale_and_offset(df,header,append_to_df=False):
    '''Function takes as input the dataframe output of raw_to_df and the laspy header file.
       Output is a nx3 dataframe with adjusted X,Y, and Z coordinates, from the formula: 
       X_adj = X*X_scale + X_offset.
       Brooklyn LiDAR readings appear to be in feet, and use NAVD 88 in the vertical and 
       New York Long Island State Plane Coordinate System NAD 33 in the horizontal.'''
    offset = header.offset
    scale = header.scale
    scaled_xyz = df[['X','Y','Z']]*scale + offset
    if append_to_df:
        df['x_scaled'] = scaled_xyz['X']
        df['y_scaled'] = scaled_xyz['Y']
        df['z_scaled'] = scaled_xyz['Z'] 
        return df
    else:
        return scaled_xyz

def create_df_pickle(file_dir,filename,column_names):
    inFile = File(file_dir+filename, mode='r')
    raw = inFile.get_points()
    df = raw_to_df(raw,column_names)
    df = scale_and_offset(df,inFile.header,append_to_df=True)
    pickle_name = 'las_points_'+filename[34:40]+'.pkl'
    df.to_pickle(file_dir + pickle_name)

def create_df_hd5(file_dir,filename,column_names):
    inFile = File(file_dir+filename, mode='r')
    raw = inFile.get_points()
    df = raw_to_df(raw,column_names)
    df = scale_and_offset(df,inFile.header,append_to_df=True)
    hdf_name = 'las_points_'+filename[34:40]+'.lz'
#     df.to_hdf(file_dir + hdf_name,key='df',complevel=1,complib='lzo')
    return df

# Load pickle, extract points around square, iterate
def grab_points(pt_files,file_dir,pt_x,pt_y,feet_from_point):
    size_of_square = (2*feet_from_point)**2
    square_points = pd.DataFrame()
    for pick in pt_files:
        las_points = pd.read_hdf(file_dir+pick)
        las_points['flight_id'] = pick[11:-3]
        new_square_points = las_points[ (las_points['x_scaled'] < pt_x + feet_from_point)
                &(las_points['x_scaled'] > pt_x - feet_from_point) 
                &(las_points['y_scaled'] < pt_y + feet_from_point)
                &(las_points['y_scaled'] > pt_y - feet_from_point)
              ]
        print("Point count in new square from {:s}: {:d}".format(pick,new_square_points.shape[0]))
        #pts_from_scan.append((pick,new_square_points.shape[0]))
        square_points = square_points.append(new_square_points,sort=True)

    print("Total point count in square: {:d}".format(square_points.shape[0]))
    print("Size of square: {:2.2f} sq ft".format(size_of_square))
    print("Point density: {:2.2f} points / sq ft".format(square_points.shape[0]/size_of_square))
    return square_points

## Flat surface
Identifying points in a parking lot to assess how consistently flat they are.  
Center point: 40.645789, -74.025951  
Easting - 977048.434  
Northing - 174555.792

In [4]:
df =create_df_hd5('../../Data/','T_315500_234000.laz',columns_dublin_pt_cloud)

In [9]:
df.shape

(82302919, 13)

In [7]:
df.describe()

,X,Y,Z,intensity,return_number_byte,classification_byte,scan_angle,user_data,pt_src_id,gps_time,x_scaled,y_scaled,z_scaled
count,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07,82302919.0,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07,8.230292e+07
mean,1.756931e+06,4.234703e+06,1.287429e+04,2.258420e+02,7.510773e+01,2.456181e+00,8.736389e-01,0.0,2.239856e+01,3.967786e+05,3.157569e+05,2.342347e+05,1.287429e+01
std,1.450786e+05,1.471303e+05,8.549734e+03,3.320399e+02,6.466772e+00,8.392025e-01,1.779510e+01,0.0,6.949736e+00,3.830248e+03,1.450786e+02,1.471303e+02,8.549734e+00
min,1.500000e+06,4.000000e+06,-9.959400e+04,0.000000e+00,7.300000e+01,2.000000e+00,-4.200000e+01,0.0,4.000000e+00,3.896547e+05,3.155000e+05,2.340000e+05,-9.959400e+01
25%,1.632096e+06,4.104507e+06,4.629000e+03,1.000000e+02,7.300000e+01,2.000000e+00,-1.400000e+01,0.0,1.700000e+01,3.933806e+05,3.156321e+05,2.341045e+05,4.629000e+00
50%,1.761575e+06,4.218552e+06,1.232400e+04,1.740000e+02,7.300000e+01,2.000000e+00,1.000000e+00,0.0,2.000000e+01,3.942447e+05,3.157616e+05,2.342186e+05,1.232400e+01
75%,1.884316e+06,4.366342e+06,1.894700e+04,2.950000e+02,7.300000e+01,2.000000e+00,1.600000e+01,0.0,2.800000e+01,3.999988e+05,3.158843e+05,2.343663e+05,1.894700e+01
max,1.999999e+06,4.499999e+06,3.433290e+05,6.553400e+04,2.490000e+02,4.000000e+00,3.900000e+01,0.0,3.500000e+01,4.031852e+05,3.160000e+05,2.345000e+05,3.433290e+02


In [10]:
df.to_csv("T_315500_234000.zip")

In [ ]:
# This works
for filename in filenames:
    create_df_hd5(file_dir,filename,columns_point_cloud)

In [ ]:
# Extract points within a square around the desired point

# Parking Lot
pt_x = 977037.343
pt_y = 174586.034

# Top of building
# pt_x = 977229.375
# pt_y = 174579.42

# Projects in back parking lot
# pt_x = 977458.238
# pt_y = 173302.388

# Solar panel
# pt_x = 977682.975
# pt_y = 174148.192

# Run this
feet_from_point = 2
square_points = grab_points(pt_files,file_dir,pt_x,pt_y,feet_from_point)

In [ ]:
# fit a plane to square_points via SVD
def plane_fit(square_points):
    '''
    Fits a plane via SVD to the provided points.
    Input: 
        (n x 3+) dataframe with fields x_scaled, y_scaled, and z_scaled
    Output: 
        normal vector - normal vector to plane fitted via MLS (3x1 numpy array)
        points - provided x,y,z points with zero mean (n x 3 numpy array)
        square_points - returns the dataframe with 'dist_from_plane' appended (n x 4+ dataframe)
        pts_on_plane - projection of x,y,z points onto the fitted plane (n x 3 numpy array)
    '''
    
    raw_points = np.array(square_points[['x_scaled','y_scaled','z_scaled']]).T
    points = raw_points.T - raw_points.mean(axis=1)
    svd = np.linalg.svd(points.T)
    norm_vector = np.transpose(svd[0])[2]    
    # Calculate each point's distance from the plane
    dist_from_plane = [np.dot(point,norm_vector) for point in points]

    # Project each point onto the plane
    proj_on_norm = dist_from_plane*np.array([norm_vector]).T
    pts_on_plane = points - proj_on_norm.T
    
    square_points['dist_from_plane'] = dist_from_plane
    
    return norm_vector,points,square_points,pts_on_plane

In [ ]:
def prep_square_for_plotting(square_points):
    square_points['size_num'] = 1
    square_points['x_plot'] = square_points['x_scaled'] - square_points['x_scaled'].min()
    square_points['y_plot'] = square_points['y_scaled'] - square_points['y_scaled'].min()
    square_points['z_plot'] = square_points['z_scaled'] - square_points['z_scaled'].min()
    return square_points

square_points = prep_square_for_plotting(square_points)
fig = px.scatter_3d(square_points_1, x='x_plot', y='y_plot', z='z_plot',
              color='flight_id',size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)",range=[0,2]),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

## Vertical density
Identifying point at corner of building to quantify the vertical point density.  
Center point: 	40.645854, 	-74.025299  
Easting - 977229.375  
Northing - 174579.42

In [ ]:
# Top of building
pt_x_bldg = 977229.375
pt_y_bldg = 174579.42

square_points_bldg = grab_points(pt_files,file_dir,pt_x_bldg,pt_y_bldg)

wall_face = square_points_bldg[(square_points_bldg['x_plot']<5) & 
                          (square_points_bldg['y_plot']<5) & 
                          (square_points_bldg['z_plot']<100) &
                          (square_points_bldg['z_plot']>10)]

In [ ]:
norm_vector,points,wall_face,pts_on_plane = plane_fit(wall_face)

In [ ]:
pts_on_plane_df = pd.DataFrame(pts_on_plane,columns=['x_plot','y_plot','z_plot'])
pts_on_plane_df['size_num'] = 1

fig = px.scatter_3d(pts_on_plane_df, x='x_plot', y='y_plot', z='z_plot',
              size='size_num',size_max = 8)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 #zaxis = dict(title="Vertical (feet)",range=[0,10]),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    xaxis = {"title":{"text":"Cat"}})
fig.show()

## NYC data

In [ ]:
# Load the data using laspy
file_dir = '../../Data/NYC_topo/'
filename = '975172.las'

create_df_hd5(file_dir,filename,columns_point_cloud)

In [ ]:
# Extract points within a square around the desired point
pt_x = 977037.343
pt_y = 174586.034
feet_from_point = 2
nyc_file_dir = '../../Data/NYC_topo/'
nyc_pt_file = ['las_points_NYC_975172.lz']
square_points_1 = grab_points(nyc_pt_file,nyc_file_dir,pt_x,pt_y,feet_from_point)

In [ ]:
# Read the .lz file
nyc = pd.read_hdf(nyc_file_dir+nyc_pt_file[0])

In [ ]:
# Time spread for nyc flight
(nyc['gps_time'].max() - nyc['gps_time'].min())/(60*60*24)

In [ ]:
def label_returns(las_df):
    '''
    Parses the flag_byte into number of returns and return number, adds these fields to las_df.
    Input - las_df - dataframe from .laz or .lz file
    Output - first_return_df - only the first return points from las_df.
           - las_df - input dataframe with num_returns and return_num fields added 
    '''
    
    las_df['num_returns'] = np.floor(las_df['flag_byte']/16).astype(int)
    las_df['return_num'] = las_df['flag_byte']%16
    first_return_df = las_df[las_df['return_num']==1]
    first_return_df = first_return_df.reset_index(drop=True)
    return first_return_df, las_df
_,nyc = label_returns(nyc)

In [ ]:
# Portion of points that are first returns
sum(nyc['return_num']==1)/nyc.shape[0]

In [ ]:
# Access the NYC header file
inFile = File(nyc_file_dir+'975172.las', mode='r')

In [ ]:
# First bit of Global Encoding indicates the gps time model: 0 for GPS Week Time, 
# 1 for GPS Adj Standard Time (Standard Time - 1e9)
# The origin of standard GPS Time is defined as midnight of the morning of January 6, 1980.
print(inFile.header.global_encoding%2)

## Plotting charts from previous updates

In [ ]:
norm_vector,points,square_points,_ = plane_fit(square_points)

# Add distance from flat plane with norm (x,y,z) = (0,0,1)
square_points['dist_from_flat']=np.array([np.dot(point,np.array([0,0,1])) for point in points])

# remove data points >5 feet below plane.
outliers = square_points[square_points['dist_from_plane']<-5].index
square_points = square_points.drop(outliers)

In [ ]:
def plot_scan_angle_dist_from_plane(df,distance_metric):
    x = abs(df['scan_angle'])*.006
    y = df[distance_metric]
    plt.figure(figsize=(15,15))
    plt.plot(x,y,'xb')
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x,p(x),"r--")
    plt.xlabel("Scan angle (degrees)")
    plt.ylabel("Point distance from plane")
    print("y={:2.8f}x+{:2.8f}".format(z[0],z[1]))
    plt.title("Scan Angle vs Distance to Fitted Plane")
plot_scan_angle_dist_from_plane(square_points,'dist_from_plane')

In [ ]:
plt.plot(range(len(square_points)),square_points['scan_angle'],'x')

In [ ]:
plot_scan_angle_dist_from_plane(square_points,'dist_from_flat')

In [ ]:
# Chart from slides showing points per run
labels = [pt[0][11:-4] for pt in pts_from_scan]
num_points = [pt[1]+.01 for pt in pts_from_scan]
plt.figure(figsize=(25,20))
plt.bar(labels,num_points,)
plt.xticks(rotation=45,fontsize=20)
plt.yticks(np.arange(0, max(num_points), step=(max(num_points)/10)),fontsize=20)
plt.ylabel("Number of points from run",fontsize=20)
plt.xlabel("Run ID",fontsize=20)